In [78]:
import pandas as pd 


In [79]:
dftesting = pd.read_pickle(r'C:\Users\SOOQ ELASER\movie_recomendation_collaborative_filtering\artifact\testing_data1.pkl')

In [89]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the pre-trained model from the pickle file
with open(r'C:\Users\SOOQ ELASER\movie_recomendation_collaborative_filtering\models\bert4rec_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Specify the path to the test dataset
test_data_path = r'C:\Users\SOOQ ELASER\movie_recomendation_collaborative_filtering\artifact\testing_data1.pkl' 

# Load the test dataset
test_df = pd.read_pickle(test_data_path)

# Ensure data is sorted chronologically if needed
test_df['datetime'] = pd.to_datetime(test_df['datetime'])
test_df = test_df.sort_values(by=['userId', 'datetime'])

# Function to create sequences of movies for each user (same as during training)
def create_sequences(df, sequence_length=50):
    user_sequences = []
    user_ids = df['userId'].unique()
    
    for user_id in user_ids:
        user_data = df[df['userId'] == user_id]
        movie_sequence = user_data['movieId'].values
        
        for i in range(len(movie_sequence) - sequence_length):
            user_sequences.append((movie_sequence[i:i + sequence_length], 
                                   list(range(sequence_length))))
    
    return user_sequences

# Create sequences from the test data
test_sequences = create_sequences(test_df)

# Prepare the test data for model evaluation
def prepare_data(sequences, sequence_length=50):
    movie_sequences = [seq[0] for seq in sequences]
    position_sequences = [seq[1] for seq in sequences]
    next_movie = [seq[0][-1] for seq in sequences]
    
    movie_sequences_padded = pad_sequences(movie_sequences, padding='post', maxlen=sequence_length)
    position_sequences_padded = pad_sequences(position_sequences, padding='post', maxlen=sequence_length)
    
    return np.array(movie_sequences_padded), np.array(position_sequences_padded), np.array(next_movie)

# Prepare the test tensors
test_movie_seq, test_position_seq, test_next_movie = prepare_data(test_sequences)
test_movie_seq_tensor = tf.convert_to_tensor(test_movie_seq, dtype=tf.int32)
test_position_seq_tensor = tf.convert_to_tensor(test_position_seq, dtype=tf.int32)
test_next_movie_tensor = tf.convert_to_tensor(test_next_movie, dtype=tf.int32)

# Evaluate the model
evaluation_metrics = model.evaluate(
    [test_movie_seq_tensor, test_position_seq_tensor], 
    test_next_movie_tensor
)

print("Model Evaluation Results:", evaluation_metrics)


AttributeError: Can't get attribute 'BERT4Rec' on <module '__main__'>

In [80]:
dftesting.head()


,movieId,title,genres,userId,datetime,movie_sequence,genre_sequence,datetime_sequence,title_sequence
131626,114818.0,108,6,11248.0,2015-01-06 13:57:40,"[76755.0, 76755.0, 76755.0, 76755.0, 76755.0, ...","[65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 6...","[2011-07-12 19:10:21, 2011-07-12 19:10:21, 201...","[86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 8..."
44208,32460.0,61,7,370.0,2008-06-19 20:16:53,"[32460.0, 32460.0, 32460.0, 32460.0, 32460.0, ...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[2008-06-19 20:16:53, 2008-06-19 20:16:53, 200...","[61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 6..."
99120,127319.0,120,65,3358.0,2015-03-29 06:28:21,"[127323.0, 127323.0, 127323.0, 127323.0, 12732...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2015-02-07 19:44:48, 2015-02-07 19:44:48, 201...","[126, 126, 126, 126, 126, 126, 126, 126, 126, ..."
90126,60365.0,39,27,1741.0,2012-07-04 16:06:16,"[6037.0, 6037.0, 6037.0, 6037.0, 6037.0, 6037....","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...","[2003-01-19 07:00:33, 2003-01-19 07:00:33, 200...","[109, 109, 109, 109, 109, 109, 109, 109, 109, ..."
10089,80846.0,24,62,129.0,2011-03-06 23:14:10,"[80094.0, 80094.0, 80094.0, 80094.0, 80094.0, ...","[63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 6...","[2011-01-08 16:42:21, 2011-01-08 16:42:21, 201...","[63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 6..."


In [81]:
dftesting.shape


(28773, 9)

In [82]:
dftraining = pd.read_pickle(r'C:\Users\SOOQ ELASER\movie_recomendation_collaborative_filtering\artifact\training_data1.pkl')


In [83]:
dftraining.head()


,movieId,title,genres,userId,datetime,movie_sequence,genre_sequence,datetime_sequence,title_sequence
109333,81140.0,1,62,4450.0,2014-08-08 04:58:07,"[6825.0, 6825.0, 6825.0, 6825.0, 6825.0, 6825....","[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 3...","[2004-01-17 01:59:42, 2004-01-17 01:59:42, 200...","[94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 9..."
65022,72224.0,43,31,739.0,2010-03-22 02:41:56,"[72224.0, 72224.0, 72224.0, 72224.0, 72224.0, ...","[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 3...","[2010-03-22 02:41:56, 2010-03-22 02:41:56, 201...","[43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 4..."
56562,91978.0,71,48,370.0,2013-03-30 14:58:41,"[32460.0, 32460.0, 32460.0, 32460.0, 32460.0, ...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[2008-06-19 20:16:53, 2008-06-19 20:16:53, 200...","[61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 6..."
119782,113501.0,3,5,10164.0,2014-12-02 04:28:57,"[113501.0, 113501.0, 113501.0, 113501.0, 11350...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[2014-12-02 04:28:57, 2014-12-02 04:28:57, 201...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
24127,1502.0,60,57,359.0,2004-06-14 21:09:23,"[3192.0, 3192.0, 3192.0, 3192.0, 3192.0, 3192....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[2003-10-26 16:08:07, 2003-10-26 16:08:07, 200...","[117, 117, 117, 117, 117, 117, 117, 117, 117, ..."


In [84]:
dftraining.shape


(115092, 9)

In [85]:
dftraining.isnull().sum()


movieId              0
title                0
genres               0
userId               0
datetime             0
movie_sequence       0
genre_sequence       0
datetime_sequence    0
title_sequence       0
dtype: int64

In [86]:
dftesting.isnull().sum()


movieId              0
title                0
genres               0
userId               0
datetime             0
movie_sequence       0
genre_sequence       0
datetime_sequence    0
title_sequence       0
dtype: int64

In [87]:
dftraining.duplicated().sum()

TypeError: unhashable type: 'list'

In [61]:



dftraining['datetime'] = pd.to_datetime(dftraining['datetime'])


dftraining = dftraining.drop(columns=['rating', 'tagId', 'relevance'])

# Sort the dataset by 'userId' and 'datetime' to ensure correct temporal order
dftraining = dftraining.sort_values(by=['userId', 'datetime'])

# Group by 'userId' and create sequences of 'movieId' and 'genres'
user_sequences_with_details = dftraining.groupby('userId').agg(
    movie_sequence=('movieId', lambda x: list(x)),
    genre_sequence=('genres', lambda x: list(x)),
    datetime=('datetime', lambda x: list(x)),  
    title=('title', lambda x: list(x))         
).reset_index()

# Merge the sequence details with the original DataFrame
dffinal = pd.merge(dftraining, user_sequences_with_details[['userId', 'movie_sequence', 'genre_sequence']], on='userId', how='left')



In [62]:
dffinal.head()

,movieId,title,genres,userId,datetime,movie_sequence,genre_sequence
0,27866.0,55,52,65.0,2011-05-09 16:05:59,"[27866.0, 27866.0, 27866.0, 27866.0, 27866.0, ...","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5..."
1,27866.0,55,52,65.0,2011-05-09 16:05:59,"[27866.0, 27866.0, 27866.0, 27866.0, 27866.0, ...","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5..."
2,27866.0,55,52,65.0,2011-05-09 16:05:59,"[27866.0, 27866.0, 27866.0, 27866.0, 27866.0, ...","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5..."
3,27866.0,55,52,65.0,2011-05-09 16:05:59,"[27866.0, 27866.0, 27866.0, 27866.0, 27866.0, ...","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5..."
4,27866.0,55,52,65.0,2011-05-09 16:05:59,"[27866.0, 27866.0, 27866.0, 27866.0, 27866.0, ...","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5..."


In [63]:
dffinal.shape

(115092, 7)

In [64]:
dffinal.duplicated().sum()

TypeError: unhashable type: 'list'

In [67]:
dffinal.dtypes

movieId                  float64
title                      int64
genres                     int64
userId                   float64
datetime          datetime64[ns]
movie_sequence            object
genre_sequence            object
dtype: object